<a href="https://colab.research.google.com/github/keymemory/DYC_Machine_Learning/blob/main/ML_day6_basic_tranfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 준비하기


In [ ]:
from tensorflow.keras.datasets import cifar10
import numpy as np

# 데이터 다운로드
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 평균과 표준편차는 채널별로 구해줍니다.
x_mean = np.mean(x_train, axis = (0, 1, 2))
x_std = np.std(x_train, axis = (0, 1, 2))

# 표준화 전처리
x_train = (x_train - x_mean) / x_std
x_test = (x_test - x_mean) / x_std


from sklearn.model_selection import train_test_split

# 학습/검정 데이터 나누기
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.3)
print('data ready~')


170500096/170498071 [==============================] - 7s 0us/step
data ready~


전이 학습 사용하기
이미지 제너레이터 설정하기


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip = True,
                                   zoom_range = 0.2,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   rotation_range = 30,
                                   fill_mode = 'nearest')

val_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow(x_train, y_train,
                                    batch_size = batch_size)

val_generator = val_datagen.flow(x_val, y_val,
                                batch_size = batch_size)



사전 학습된 모델 사용하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import VGG16

# imagenet을 학습한 모델을 불러옵니다.
vgg16 = VGG16(weights = 'imagenet', input_shape = (32, 32, 3), include_top = False)
vgg16.summary()


58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

모델 구성 및 학습하기


In [ ]:
model = Sequential()
model.add(vgg16)
# 분류기를 직접 정의합니다.
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation = 'softmax'))

# model.summary() # 모델의 구조를 확인하세요!


모델 설정하기


In [ ]:
model.compile(optimizer = Adam(1e-4),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['acc'])



모델 학습하기


In [ ]:
def get_step(train_len, batch_size):
    if(train_len % batch_size > 0):
        return train_len // batch_size + 1
    else:
        return train_len // batch_size

history = model.fit(train_generator,
                    epochs = 100,
                    steps_per_epoch = get_step(len(x_train), batch_size),
                    validation_data = val_generator,
                    validation_steps = get_step(len(x_val), batch_size))


Epoch 1/100
1094/1094 [==============================] - 28s 25ms/step - loss: 1.0698 - acc: 0.0999 - val_loss: 0.7204 - val_acc: 0.1115
Epoch 2/100
1094/1094 [==============================] - 27s 25ms/step - loss: 0.7756 - acc: 0.0986 - val_loss: 0.6256 - val_acc: 0.0937
Epoch 3/100
1094/1094 [==============================] - 27s 25ms/step - loss: 0.6747 - acc: 0.0997 - val_loss: 0.5860 - val_acc: 0.1043
Epoch 4/100
1094/1094 [==============================] - 27s 25ms/step - loss: 0.6043 - acc: 0.0999 - val_loss: 0.5580 - val_acc: 0.1151
Epoch 5/100
1094/1094 [==============================] - 27s 25ms/step - loss: 0.5548 - acc: 0.0986 - val_loss: 0.6021 - val_acc: 0.0905
Epoch 6/100
1094/1094 [==============================] - 27s 25ms/step - loss: 0.5097 - acc: 0.0996 - val_loss: 0.5136 - val_acc: 0.1000
Epoch 7/100
1094/1094 [==============================] - 27s 25ms/step - loss: 0.4742 - acc: 0.0996 - val_loss: 0.4919 - val_acc: 0.1065
Epoch 8/100
1094/1094 [==================